In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from typing import Callable, Dict, List, Optional, Tuple, Union

import mne
from mne.io.edf.edf import RawEDF

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test
DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown


In [2]:
from utils import read_data
from utils import filter_band_raw
from utils import butter_bandpass_filter
from utils import rref_REST

DEBUG:root:test


In [3]:
# Load data, filter on 0.1-100 [Hz] and rref REST
raw = read_data("../data/h01.edf")
raw_filtered = filter_band_to_raw(raw, [0.1, 100])
raw_REST = rref_REST(raw_filtered)

Extracting EDF parameters from /home/camilojd/Universidad/Primavera_2020/EL7006/dynamicinfo-eeg/data/h01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 231249  =      0.000 ...   924.996 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 8251 samples (33.004 sec)



In [12]:
# Butter filter (order 2) on mean of Fp1 and Fp2 EEG signals
ocular_virtual = butter_bandpass_filter(
    data=np.mean(np.concatenate([raw_REST["Fp1"][0], raw_REST["Fp2"][0]]), axis=0), 
    band=[0.1, 5], 
    fs=raw.info["sfreq"], 
    order=2
)
ocular_virtual.shape

(231250,)

In [5]:
# Decomposition ICA
raw_ica = raw_REST.copy()
ica = mne.preprocessing.ICA()
ica.fit(raw_ica)

Fitting ICA to data using 19 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Selecting all PCA components: 19 components
Fitting ICA took 3.9s.


<ICA | raw data decomposition, fit (fastica): 231250 samples, 19 components, channels used: "eeg">

In [6]:
# Concatenate ocular_virtual with ICA decompositioned data 
virtual_and_ica = np.concatenate([np.expand_dims(ocular_virtual, 0), raw_ica.get_data()], axis=0)
cov = np.cov(virtual_and_ica)

In [13]:
cov

array([[ 5.77375365e-11,  5.23758490e-11,  4.45689231e-12,
        -1.45981004e-11, -1.78138840e-11, -1.65204162e-11,
         3.49301618e-11,  7.67235684e-12, -1.40624529e-11,
        -1.77252484e-11, -1.59630602e-11, -3.35481550e-12,
        -1.33130629e-11, -1.54280940e-11, -1.30870625e-12,
        -1.17985055e-11, -1.53988692e-11, -4.58629874e-13,
        -1.28847184e-11, -1.67313526e-11],
       [ 5.23758490e-11,  1.30540000e-10,  5.16778777e-11,
         2.36350855e-12, -4.29944539e-11, -1.14218814e-10,
         8.62392830e-11,  3.51658693e-11, -8.27779189e-12,
        -5.76046518e-11, -1.11406352e-10,  3.76688252e-11,
         5.08659497e-12, -2.55297722e-11,  2.72604919e-11,
        -7.19400840e-13, -3.27841155e-11,  3.95372960e-11,
         8.90694525e-12, -4.01590634e-11],
       [ 4.45689231e-12,  5.16778777e-11,  7.22709555e-11,
         2.76785111e-11, -6.05325008e-12, -7.85115173e-11,
         3.34087240e-11,  6.43934541e-12, -1.53551125e-12,
        -3.24961113e-11, -8.0

In [7]:
np.diag(cov)

array([5.77375365e-11, 1.30540000e-10, 7.22709555e-11, 3.36750945e-11,
       5.50598081e-11, 2.28475340e-10, 9.26489344e-11, 7.55460281e-11,
       3.20795494e-11, 6.34099984e-11, 2.08200704e-10, 6.55025381e-11,
       3.19862478e-11, 3.43714512e-11, 4.63459618e-11, 3.11419703e-11,
       2.90225085e-11, 5.85971535e-11, 5.71010471e-11, 4.39612498e-11])

In [8]:
#%matplotlib notebook
#ica.plot_sources(raw_ica)
#raw_REST.plot()
#plt.show()

In [9]:
#ica.plot_components(inst=raw_REST)

In [10]:
#ica.apply(raw)
#raw.plot()
#plt.show()

In [11]:
#ica.plot_overlay(raw)
#plt.show()